In [2]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64
import datetime
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animalShelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "accuser"
password = "jay"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.A([
        html.Img(
        src='data:image/png;base64,{}'.format(encoded_image.decode()),
        style = {'height': '10%', 'width':'25%'})
    ], href='https://www.snhu.edu'),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard by Justin Chevalier'))),
    html.Hr(),
    html.Div(className='row',
             style = {'display':'flex'},
             children = [
                 html.Button(id ='submit-button-one',n_clicks_timestamp = 0, children='Water Rescue'),
                 html.Button(id ='submit-button-two',n_clicks_timestamp = 0, children='Mountain Rescue'),
                 html.Button(id ='submit-button-three', n_clicks_timestamp = 0, children = 'Disaster Rescue'),
                 html.Button(id = 'submit-button-four', n_clicks_timestamp = 0, children = 'Reset')
             ]
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
 
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
         return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]




    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('submit-button-one', 'n_clicks_timestamp'),
               Input('submit-button-two', 'n_clicks_timestamp'),
               Input('submit-button-three', 'n_clicks_timestamp'),
               Input('submit-button-four', 'n_clicks_timestamp'),])
def on_click(bt1, bt2, bt3, bt4):
    if (int(bt1) == 0 and int(bt2) == 0 and int(bt3) == 0 and int(bt4) == 0):
        df = pd.DataFrame(list(shelter.read_all({'animal_type': 'Dog'})))
        
    if (int(bt1) > int(bt2) and int(bt1) > int(bt3) and int(bt1) > int(bt4)):
        df = pd.DataFrame(list(shelter.read_all({'animal_type':'Dog',
                                                 'breed':{'$in':[
                                                     'Labrador Retriever Mix', 'Chesapeake Bay Retriever',
                                                     'Newfoundland']},
                                                "sex_upon_outcome" : 'Intact Female',
                                                'age_upon_outcome_in_weeks':{'$gt': 26, '$lt':156}})))
    if (int(bt2) > int(bt1) and int(bt2) > int(bt3) and int(bt2) > int(bt4)):
        df = pd.DataFrame(list(shelter.read_all({'animal_type':'Dog',
                                                 'breed':{'$in':[
                                                     'German Shepherd', 'Alaskan Malamute',
                                                     'Old English Sheepdog','Siberian Husky', 'Rottweiler']},
                                                'sex_upon_outcome': 'Intact Male',
                                                'age_upon_outcome_in_weeks':{'$gt': 26, '$lt':156}})))
    if (int(bt3) > int(bt1) and int(bt3) > int(bt2) and int(bt3) > int(bt4)):
        df = pd.DataFrame(list(shelter.read_all({'animal_type':'Dog',
                                                 'breed':{'$in':[
                                                     'Doberman Pinscher', 'German Shepherd',
                                                     'Golden Retriever', 'Bloodhound', 'Rottweiler']},
                                                'sex_upon_outcome': 'Intact Male',
                                                'age_upon_outcome_in_weeks':{'$gt': 20, '$lt':300}})))  
        
    if (int(bt4) > int(bt1) and int(bt4) > int(bt2) and int(bt4) > int(bt3)):
        df = pd.DataFrame(list(shelter.read_all({'animal_type': 'Dog'})))
        
     
        
        


        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        reutrn (data,columns)
        








@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
        dff = pd.DataFrame.from_dict(viewData)
        fig = px.pie(dff, names='breed')
        return fig
     
    


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dl.Map(style =  {'width': '1000px', 'height': '500px'},
               center = [30.75, -97.48],
               children = [dl.TileLayer(id = "base-layer-id"),
                           dl.Marker(position = [30.75, -97.48],
                           children = [dl.Tooltip(dff.iloc[0,4]),
                            dl.Popup([
                                html.H1("Animal Name"),
                                html.P(dff.iloc[0,9])
                            ])          ]         )])
    ]




app

Successful connection
